# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_ky import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Save file
path = "../WeatherPy/output_data/cities.csv"
city_df = pd.read_csv(path)

city_df.head()

,City,Lat,Lng,Country,Max Temp,Humidity,Wind Speed,Cloudiness,Date
0,corinto,37.9407,22.9573,GR,60.69,77,3.51,74,1635649605
1,mwanza,-2.5167,32.9000,TZ,72.01,74,6.29,94,1635649606
2,yar-sale,66.8333,70.8333,RU,4.68,90,14.14,16,1635649606
3,busselton,-33.6500,115.3333,AU,67.08,61,6.49,98,1635649606
4,sibu,2.3000,111.8167,MY,85.55,72,3.80,68,1635649606


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
heatmap = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [5]:
# Add Heatmap layer to map
heat = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
        
heatmap.add_layer(heat)
heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities to fit weather conditions
clouds = city_df['Cloudiness']
temp = city_df['Max Temp']
wind = city_df['Wind Speed']

fit_df = city_df.loc[(temp >= 70) & (temp <= 80)]
fit_df = fit_df.loc[(clouds <= 10)]
fit_df = fit_df.loc[(clouds == 0)]

In [7]:
# Drop any rows will null values
fit_df.dropna()

,City,Lat,Lng,Country,Max Temp,Humidity,Wind Speed,Cloudiness,Date
93,carnarvon,-24.8667,113.6333,AU,78.94,47,14.34,0,1635649631
107,geraldton,-28.7667,114.6000,AU,75.61,43,11.50,0,1635649635
154,riyadh,24.6877,46.7219,SA,71.74,43,5.50,0,1635649625
158,empalme,27.9667,-110.8167,MX,71.92,53,1.72,0,1635649649
245,salalah,17.0151,54.0924,OM,77.09,47,6.91,0,1635649454
270,chakia,26.4167,85.0500,IN,74.80,58,5.14,0,1635649677
293,bela,25.9333,81.9833,IN,73.63,51,4.29,0,1635649684
396,frontera,26.9333,-101.4500,MX,72.36,26,4.92,0,1635649712
429,nagaur,27.2000,73.7333,IN,75.20,25,8.63,0,1635649454
490,saint-pierre,-21.3393,55.4781,RE,73.56,68,5.75,0,1635649736


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Add Hotel Name Column and rearrange for readability
fit_df['Hotel Name'] = ''
fit_df['Hotel Address'] = ''
hotel_df = fit_df[['City', 'Country','Hotel Name', 'Hotel Address',
                     'Humidity', 'Lat', 'Lng', 'Max Temp', 'Wind Speed']]

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\Viola\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [9]:
hotel_df.loc[hotel_df['Hotel Name'] != '']

,City,Country,Hotel Name,Hotel Address,Humidity,Lat,Lng,Max Temp,Wind Speed
93,carnarvon,AU,The Gascoyne Hotel,"57 Olivia Terrace, Carnarvon",47,-24.8667,113.6333,78.94,14.34
107,geraldton,AU,Ocean Centre Hotel,"Cathedral Ave &, Foreshore Dr, Geraldton",43,-28.7667,114.6000,75.61,11.50
154,riyadh,SA,Four Seasons Hotel Riyadh At Kingdom Center,"Center Point, 2239 King Fahd Rd, Riyadh",43,24.6877,46.7219,71.74,5.50
158,empalme,MX,Hotel Empalme Inn,"Cdad. Obregón - Guaymas, Luis Donaldo Colosio,...",53,27.9667,-110.8167,71.92,1.72
245,salalah,OM,Salalah Gardens Hotel,Al Rabat Street، Salalah,47,17.0151,54.0924,77.09,6.91
270,chakia,IN,Hotel Rajmahal International,"Hospital Rd, Chakia",58,26.4167,85.0500,74.80,5.14
293,bela,IN,Hotel Rajvilla,"479, Chowk, Jail Rd, Bela Pratapgarh",51,25.9333,81.9833,73.63,4.29
396,frontera,MX,"Holiday Inn Monclova, an IHG Hotel","Blvd Harold R. Pape 200, Guadalupe, Monclova",26,26.9333,-101.4500,72.36,4.92
429,nagaur,IN,Mahindra Hotel & Restaurant Nagaur,"Bikaner Bypass, Jodhpur Road, National Highway...",25,27.2000,73.7333,75.20,8.63
490,saint-pierre,RE,Hôtel Le Saint-Pierre,"51 Av. des Indes, Saint-Pierre",68,-21.3393,55.4781,73.56,5.75


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))